In [1]:
import cv2
import numpy as np
from skimage import measure
import math
import os
from IPython.display import display, Image
import tifffile
import glob

In [2]:
def detect_measure_and_highlight_circle(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_crop = gray [:-100]
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray_crop, (1, 1), 0)
    
    # Use Hough Circle Transform to detect circles
    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=50,
        param1=50,
        param2=30,
        minRadius=0,
        maxRadius=0
    )
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        
        # Get the first detected circle
        x, y, r = circles[0][0]
        
        # Calculate diameter
        diameter = 2 * r
        
        # Create a binary mask of the circle
        mask = np.zeros(gray.shape, dtype=np.uint8)
        cv2.circle(mask, (x, y), r, 255, -1)
        
        # Find contours of the mask
        contours = measure.find_contours(mask, 0.5)
        
        if contours:
            contour = contours[0]
            
            # Calculate area and perimeter
            area = cv2.contourArea(contour.astype(np.float32))
            perimeter = cv2.arcLength(contour.astype(np.float32), True)
            
            # Calculate circularity
            circularity = 4 * math.pi * area / (perimeter ** 2)
            
            # Calculate roundness
            _, (major_axis, minor_axis), _ = cv2.fitEllipse(contour.astype(np.float32))
            roundness = min(4 * area / (math.pi * major_axis ** 2), 1.0)
            
            # Draw the circle on the original image
            cv2.circle(image, (x, y), r, (0, 255, 0), 2)
            cv2.circle(image, (x, y), 2, (0, 0, 255), 3)
            
            # Add text with measurements
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, f"Diameter: {diameter}px", (10, 30), font, 0.7, (255, 255, 255), 2)
            cv2.putText(image, f"Circularity: {circularity:.4f}", (10, 60), font, 0.7, (255, 255, 255), 2)
            cv2.putText(image, f"Roundness: {roundness:.4f}", (10, 90), font, 0.7, (255, 255, 255), 2)
            
            return image, diameter, circularity, roundness
    
    return image, None, None, None

In [3]:
def save_analysis_image(original_path, analyzed_image):
    # Get the directory and filename of the original image
    directory, filename = os.path.split(original_path)
    
    # Create the 'analysis' folder if it doesn't exist
    analysis_folder = os.path.join(directory, 'analysis')
    os.makedirs(analysis_folder, exist_ok=True)
    
    # Create the new filename with 'ana' appended and .tiff extension
    name, _ = os.path.splitext(filename)
    new_filename = f"{name}_ana.tiff"
    
    # Save the analyzed image in the 'analysis' folder as TIFF
    save_path = os.path.join(analysis_folder, new_filename)
    tifffile.imwrite(save_path, cv2.cvtColor(analyzed_image, cv2.COLOR_BGR2RGB))
    
    return save_path

In [4]:
def process_images_in_directory(directory):
    # Get all TIFF files in the directory
    tiff_files = glob.glob(os.path.join(directory, '*.tiff')) + glob.glob(os.path.join(directory, '*.tif'))
    
    results = []
    
    for image_path in tiff_files:
        # Read the image
        img = cv2.imread(image_path)
        
        # Process the image
        result_image, diameter, circularity, roundness = detect_measure_and_highlight_circle(img)
        
        if diameter is not None:
            # Save the analyzed image
            saved_path = save_analysis_image(image_path, result_image)
            
            results.append({
                'filename': os.path.basename(image_path),
                'diameter': diameter,
                'circularity': circularity,
                'roundness': roundness,
                'saved_path': saved_path
            })
            
            print(f"Processed: {os.path.basename(image_path)}")
            print(f"  Diameter: {diameter} pixels")
            print(f"  Circularity: {circularity:.4f}")
            print(f"  Roundness: {roundness:.4f}")
            print(f"  Saved to: {saved_path}")
            print()
        else:
            print(f"No circle detected in: {os.path.basename(image_path)}")
    
    return results

In [5]:
directory_path = "/Users/sichen/Box Sync/projects/2021_ECRP/data/202409_Xe_spot_milling/"  # Update this path
results = process_images_in_directory(directory_path)

Processed: 30kV-100nA_008.tif
  Diameter: 170 pixels
  Circularity: 0.8920
  Roundness: 1.0000
  Saved to: /Users/sichen/Box Sync/projects/2021_ECRP/data/202409_Xe_spot_milling/analysis/30kV-100nA_008_ana.tiff

Processed: 30kV-100nA_009.tif
  Diameter: 168 pixels
  Circularity: 0.8948
  Roundness: 1.0000
  Saved to: /Users/sichen/Box Sync/projects/2021_ECRP/data/202409_Xe_spot_milling/analysis/30kV-100nA_009_ana.tiff

Processed: 30kV-1nA-spot4_009.tif
  Diameter: 218 pixels
  Circularity: 0.8924
  Roundness: 1.0000
  Saved to: /Users/sichen/Box Sync/projects/2021_ECRP/data/202409_Xe_spot_milling/analysis/30kV-1nA-spot4_009_ana.tiff

Processed: 30kV-1nA-spot1_006.tif
  Diameter: 546 pixels
  Circularity: 0.8963
  Roundness: 1.0000
  Saved to: /Users/sichen/Box Sync/projects/2021_ECRP/data/202409_Xe_spot_milling/analysis/30kV-1nA-spot1_006_ana.tiff

Processed: 30nA-spot5_026.tif
  Diameter: 234 pixels
  Circularity: 0.8946
  Roundness: 1.0000
  Saved to: /Users/sichen/Box Sync/projects/2

Processed: 30kV-100nA_010.tif
  Diameter: 150 pixels
  Circularity: 0.8895
  Roundness: 1.0000
  Saved to: /Users/sichen/Box Sync/projects/2021_ECRP/data/202409_Xe_spot_milling/analysis/30kV-100nA_010_ana.tiff

Processed: 30kV-100nA_011.tif
  Diameter: 142 pixels
  Circularity: 0.8910
  Roundness: 1.0000
  Saved to: /Users/sichen/Box Sync/projects/2021_ECRP/data/202409_Xe_spot_milling/analysis/30kV-100nA_011_ana.tiff

Processed: 30nA-spot2_023.tif
  Diameter: 236 pixels
  Circularity: 0.8967
  Roundness: 1.0000
  Saved to: /Users/sichen/Box Sync/projects/2021_ECRP/data/202409_Xe_spot_milling/analysis/30nA-spot2_023_ana.tiff

Processed: 30kV-1nA-spot5_010.tif
  Diameter: 436 pixels
  Circularity: 0.8962
  Roundness: 1.0000
  Saved to: /Users/sichen/Box Sync/projects/2021_ECRP/data/202409_Xe_spot_milling/analysis/30kV-1nA-spot5_010_ana.tiff

Processed: 30nA-spot3_024.tif
  Diameter: 236 pixels
  Circularity: 0.8967
  Roundness: 1.0000
  Saved to: /Users/sichen/Box Sync/projects/2021_ECRP

In [20]:
print("Summary of processed images:")
for result in results:
    print(f"File: {result['filename']}")
    print(f"  Diameter: {result['diameter']} pixels")
    print(f"  Circularity: {result['circularity']:.4f}")
    print(f"  Roundness: {result['roundness']:.4f}")
    print(f"  Analyzed image: {result['saved_path']}")
    print()

    # Display the first few processed images
    if results.index(result) < 3:  # Limit to first 3 images to avoid cluttering the notebook
        display(Image(filename=result['saved_path']))

# Explicitly save the notebook
from IPython.display import Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))
print("Notebook saved.")



Summary of processed images:
File: 30nA-spot1_022.tif
  Diameter: 242 pixels
  Circularity: 0.8930
  Roundness: 1.0000
  Analyzed image: /Users/sichen/Box Sync/projects/2021_ECRP/data/202409_Xe_spot_milling/analysis/30nA-spot1_022_ana.tiff



ValueError: Cannot embed the 'tiff' image format